In [7]:
from tensoraerospace.agent.pid import PID
import numpy as np
from ray import train, tune

from tensoraerospace.envs.f16.linear_longitudial import LinearLongitudinalF16
from tensoraerospace.utils import generate_time_period, convert_tp_to_sec_tp
from tensoraerospace.signals.standart import unit_step

import gymnasium as gym 
from tensoraerospace.benchmark.function import overshoot, settling_time, static_error

(raylet) [2024-08-15 03:17:41,010 E 79956 2131403] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-08-15_03-16-19_045046_79511 is over 95% full, available space: 10041540608; capacity: 494384795648. Object creation will fail if spilling is required.
(raylet) [2024-08-15 03:17:51,019 E 79956 2131403] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-08-15_03-16-19_045046_79511 is over 95% full, available space: 10044186624; capacity: 494384795648. Object creation will fail if spilling is required.


In [8]:
dt = 0.01  # Дискретизация
tp = generate_time_period(tn=20, dt=dt) # Временной периуд
tps = convert_tp_to_sec_tp(tp, dt=dt)
number_time_steps = len(tp) # Количество временных шагов
reference_signals = np.reshape(unit_step(degree=5, tp=tp, time_step=10, output_rad=True), [1, -1]) # Заданный сигнал

In [19]:
def env_optimization(ki, kp, kd):
    """
    Оптимизация среды моделирования для настройки коэффициентов ПИД-регулятора.

    Эта функция оптимизирует параметры ПИД-регулятора (ки, кп, кд), используя модельную среду.
    Она оценивает качество настройки по критериям статической ошибки, перерегулирования и времени установления.

    Args:
        ki (float): Коэффициент интегральной составляющей.
        kp (float): Коэффициент пропорциональной составляющей.
        kd (float): Коэффициент дифференциальной составляющей.

    Returns:
        float: Суммарная оценка качества настройки регулятора.

    """

    # Инициализация истории и настройка параметров времени
    hist = []
    dt = 0.01
    tp = generate_time_period(tn=25, dt=dt)
    tps = convert_tp_to_sec_tp(tp, dt=dt)
    number_time_steps = len(tp)

    # Создание заданного сигнала
    reference_signals = np.reshape(unit_step(degree=5, tp=tp, time_step=10, output_rad=True), [1, -1])

    # Настройка модельной среды
    env = gym.make('LinearLongitudinalF16-v0',
               number_time_steps=number_time_steps, 
               use_reward = False,
               initial_state=[[0],[0], [0]],
               reference_signal=reference_signals,
               state_space = [ "theta", "alpha", "q",],
                output_space = ["theta", "alpha", "q", ],
                tracking_states=["alpha"])
    env.reset()

    # Инициализация ПИД-регулятора
    pid = PID(env=env, kp=kp, ki=ki, kd=kd, dt=dt)
    xt, info = env.reset()

    # Цикл моделирования среды
    for step in range(number_time_steps - 2):
        setpoint = reference_signals[0, step]
        hist.append(xt[1])
        ut = pid.select_action(setpoint, xt[1])
        xt, reward, terminated, truncated, info = env.step(np.array([ut.item()]))

    # Условие для досрочного прекращения оптимизации
    if 6 < max(hist):
        return 10000

    # Получение исходных сигналов для анализа
    system_signal_orig = env.unwrapped.model.get_state('alpha', to_deg=True)[:2500]
    control_signal_orig = np.rad2deg(reference_signals[0])[:2500]

    # Расчет оценочной функции
    return np.abs(static_error(control_signal_orig, system_signal_orig)) * 0.3 + \
           np.abs(overshoot(control_signal_orig, system_signal_orig)) ** 0.6 


In [20]:
env_optimization(1, 1, 1)

42.16761640895795

In [21]:
def easy_objective(config):
    """
    Функция цели для оптимизатора.

    Принимает конфигурацию с параметрами ПИД-регулятора и возвращает оценку качества настройки.

    Args:
        config (dict): Словарь с параметрами ПИД-регулятора (ki, kp, kd).

    """

    # Извлечение гиперпараметров из конфигурации
    ki, kp, kd = config["ki"], config["kp"], config["kd"]

    # Вычисление оценки
    intermediate_score = env_optimization(ki, kp, kd)

    # Отчет о текущем значении потерь

    train.report({"mean_loss": intermediate_score})

In [22]:
from ray.train import RunConfig, ScalingConfig

# Настройка и запуск оптимизатора
tuner = tune.Tuner(
        easy_objective,
        tune_config=tune.TuneConfig(
            metric="mean_loss",
            mode="min",
            num_samples=5000,
        ),
        run_config = RunConfig(
            verbose=False
        ),
        param_space={
            "ki": tune.uniform(-25, 25),
            "kp": tune.uniform(-25, 25),
            "kd": tune.uniform(-25, 25),
        },
    )

In [23]:
# Выполнение оптимизации
results = tuner.fit()

2024-08-15 03:20:52,703	ERROR worker.py:406 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): The worker died unexpectedly while executing this task. Check python-core-worker-*.log files for more information.
2024-08-15 03:20:52,992	ERROR worker.py:406 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): The worker died unexpectedly while executing this task. Check python-core-worker-*.log files for more information.
2024-08-15 03:21:14,269	ERROR worker.py:406 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): The worker died unexpectedly while executing this task. Check python-core-worker-*.log files for more information.
2024-08-15 03:21:14,381	ERROR worker.py:406 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): The worker died unexpectedly while executing this task. Check python-core-worker-*.log files for more information.
2024-08-15 03:21:34,074	ERROR worker.py:406 -- Unhandled error (suppress with 'RAY_IGNORE_UN

In [26]:
# Получение лучших результатов оптимизации
results.get_best_result().config

{'ki': -7.047057683739116, 'kp': 24.900324972774946, 'kd': 9.50889143291738}

In [27]:
# Сортировка и получение значений средних потерь
results.get_dataframe()['mean_loss'].sort_values()

3666     0.317434
3947     0.380368
1176     0.398261
1317     0.436818
1331     0.484023
          ...    
3422    42.235931
3421    42.235931
1551    42.235931
4380    42.235931
3751    42.235931
Name: mean_loss, Length: 5000, dtype: float64